<a href="https://colab.research.google.com/github/veer8023/veer3333/blob/main/efficeintnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import time
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from pathlib import Path
import os.path
import random
import cv2
from tensorflow.keras.applications import InceptionV3
from tensorflow import keras
from PIL import Image, ImageChops, ImageEnhance
import seaborn as sns
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard ,LearningRateScheduler
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers,models
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.utils import class_weight
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, Adamax

In [18]:
BATCH_SIZE = 32
TARGET_SIZE = (224, 224)
# Walk through each directory
dataset = "/content/drive/MyDrive/DriveApp (2)/CUB_200_2011/images"

In [19]:
image_dir = Path(dataset)

# Get filepaths and labels
filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.png')) + list(image_dir.glob(r'**/*.png'))

labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels], axis=1)
labels

0            003.Sooty_Albatross
1            003.Sooty_Albatross
2            003.Sooty_Albatross
3            003.Sooty_Albatross
4            003.Sooty_Albatross
                  ...           
11793    200.Common_Yellowthroat
11794    200.Common_Yellowthroat
11795    200.Common_Yellowthroat
11796    200.Common_Yellowthroat
11797    200.Common_Yellowthroat
Name: Label, Length: 11798, dtype: object

In [20]:
def compute_ela_cv(path, quality):
    temp_filename = 'temp_file_name.jpeg'
    SCALE = 15
    orig_img = cv2.imread(path)
    orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)

    cv2.imwrite(temp_filename, orig_img, [cv2.IMWRITE_JPEG_QUALITY, quality])

    # read compressed image
    compressed_img = cv2.imread(temp_filename)

    # get absolute difference between img1 and img2 and multiply by scale
    diff = SCALE * cv2.absdiff(orig_img, compressed_img)
    return diff


def convert_to_ela_image(path, quality):
    temp_filename = 'temp_file_name.jpeg'
    ela_filename = 'temp_ela.png'
    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)

    ela_image = ImageChops.difference(image, temp_image)

    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1

    scale = 255.0 / max_diff
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)

    return ela_image


def random_sample(path, extension=None):
    if extension:
        items = Path(path).glob(f'*.{extension}')
    else:
        items = Path(path).glob(f'*')

    items = list(items)

    p = random.choice(items)
    return p.as_posix()



In [21]:
# Separate in train and test data
train_df, test_df = train_test_split(image_df, test_size=0.2, shuffle=True, random_state=42)
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2
)

test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)
# Split the data into three categories.
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)


Found 7551 validated image filenames belonging to 200 classes.
Found 1887 validated image filenames belonging to 200 classes.
Found 2360 validated image filenames belonging to 200 classes.


In [24]:
img_size = (224, 224,3)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(list(train_images.class_indices.keys()))  # to define the number of classes in the dense layer

# create pre-trained model
base_model = tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, weights="imagenet", input_shape=img_shape, pooling='max')

model = Sequential([
    base_model,
    BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
    Dense(1024, kernel_regularizer=regularizers.l2(l=0.016), activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006), activation='relu'),

    Dense(512, kernel_regularizer=regularizers.l2(l=0.016), activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006), activation='relu'),
    Dropout(rate=0.3, seed=123),
    Dense(class_count, activation='softmax')
])

model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
#model.load_weights('./checkpoints/my_checkpoint')
model.summary()









Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional)  (None, 1536)             10783535  
                                                                 
 batch_normalization_7 (Batc  (None, 1536)             6144      
 hNormalization)                                                 
                                                                 
 dense_21 (Dense)            (None, 1024)              1573888   
                                                                 
 dense_22 (Dense)            (None, 512)               524800    
                                                                 
 dropout_14 (Dropout)        (None, 512)               0         
                                                                 
 dense_23 (Dense)            (None, 200)               102600    
                                                      

In [25]:
class MyCallback(keras.callbacks.Callback):
    def __init__(self, model, base_model, patience, stop_patience, threshold, factor, batches, initial_epoch, epochs, ask_epoch):
        super(MyCallback, self).__init__()
        self.model = model
        self.base_model = base_model
        self.patience = patience # specifies how many epochs without improvement before learning rate is adjusted
        self.stop_patience = stop_patience # specifies how many times to adjust lr without improvement to stop training
        self.threshold = threshold # specifies training accuracy threshold when lr will be adjusted based on validation loss
        self.factor = factor # factor by which to reduce the learning rate
        self.batches = batches # number of training batch to runn per epoch
        self.initial_epoch = initial_epoch
        self.epochs = epochs
        self.ask_epoch = ask_epoch
        self.ask_epoch_initial = ask_epoch # save this value to restore if restarting training
        # callback variables
        self.count = 0 # how many times lr has been reduced without improvement
        self.stop_count = 0
        self.best_epoch = 1   # epoch with the lowest loss
        self.initial_lr = float(tf.keras.backend.get_value(model.optimizer.lr)) # get the initial learning rate and save it
        self.highest_tracc = 0.0 # set highest training accuracy to 0 initially
        self.lowest_vloss = np.inf # set lowest validation loss to infinity initially
        self.best_weights = self.model.get_weights() # set best weights to model's initial weights
        self.initial_weights = self.model.get_weights()   # save initial weights if they have to get restored

    # Define a function that will run when train begins
    def on_train_begin(self, logs= None):
        msg = '{0:^8s}{1:^10s}{2:^9s}{3:^9s}{4:^9s}{5:^9s}{6:^9s}{7:^10s}{8:10s}{9:^8s}'.format('Epoch', 'Loss', 'Accuracy', 'V_loss', 'V_acc', 'LR', 'Next LR', 'Monitor','% Improv', 'Duration')
        print(msg)
        self.start_time = time.time()

    def on_train_end(self, logs= None):
        stop_time = time.time()
        tr_duration = stop_time - self.start_time
        hours = tr_duration // 3600
        minutes = (tr_duration - (hours * 3600)) // 60
        seconds = tr_duration - ((hours * 3600) + (minutes * 60))
        msg = f'training elapsed time was {str(hours)} hours, {minutes:4.1f} minutes, {seconds:4.2f} seconds)'
        print(msg)
        self.model.set_weights(self.best_weights) # set the weights of the model to the best weights

    def on_train_batch_end(self, batch, logs= None):
        acc = logs.get('accuracy') * 100 # get batch accuracy
        loss = logs.get('loss')
        msg = '{0:20s}processing batch {1:} of {2:5s}-   accuracy=  {3:5.3f}   -   loss: {4:8.5f}'.format(' ', str(batch), str(self.batches), acc, loss)
        print(msg, '\r', end= '') # prints over on the same line to show running batch count

    def on_epoch_begin(self, epoch, logs= None):
        self.ep_start = time.time()

    # Define method runs on the end of each epoch
    def on_epoch_end(self, epoch, logs= None):
        ep_end = time.time()
        duration = ep_end - self.ep_start

        lr = float(tf.keras.backend.get_value(self.model.optimizer.lr)) # get the current learning rate
        current_lr = lr
        acc = logs.get('accuracy')  # get training accuracy
        v_acc = logs.get('val_accuracy')  # get validation accuracy
        loss = logs.get('loss')  # get training loss for this epoch
        v_loss = logs.get('val_loss')  # get the validation loss for this epoch

        if acc < self.threshold: # if training accuracy is below threshold adjust lr based on training accuracy
            monitor = 'accuracy'
            if epoch == 0:
                pimprov = 0.0
            else:
                pimprov = (acc - self.highest_tracc ) * 100 / self.highest_tracc # define improvement of model progres

            if acc > self.highest_tracc: # training accuracy improved in the epoch
                self.highest_tracc = acc # set new highest training accuracy
                self.best_weights = self.model.get_weights() # training accuracy improved so save the weights
                self.count = 0 # set count to 0 since training accuracy improved
                self.stop_count = 0 # set stop counter to 0
                if v_loss < self.lowest_vloss:
                    self.lowest_vloss = v_loss
                self.best_epoch = epoch + 1  # set the value of best epoch for this epoch

            else:
                # training accuracy did not improve check if this has happened for patience number of epochs
                # if so adjust learning rate
                if self.count >= self.patience - 1: # lr should be adjusted
                    lr = lr * self.factor # adjust the learning by factor
                    tf.keras.backend.set_value(self.model.optimizer.lr, lr) # set the learning rate in the optimizer
                    self.count = 0 # reset the count to 0
                    self.stop_count = self.stop_count + 1 # count the number of consecutive lr adjustments
                    self.count = 0 # reset counter
                    if v_loss < self.lowest_vloss:
                        self.lowest_vloss = v_loss
                else:
                    self.count = self.count + 1 # increment patience counter

        else: # training accuracy is above threshold so adjust learning rate based on validation loss
            monitor = 'val_loss'
            if epoch == 0:
                pimprov = 0.0
            else:
                pimprov = (self.lowest_vloss - v_loss ) * 100 / self.lowest_vloss
            if v_loss < self.lowest_vloss: # check if the validation loss improved
                self.lowest_vloss = v_loss # replace lowest validation loss with new validation loss
                self.best_weights = self.model.get_weights() # validation loss improved so save the weights
                self.count = 0 # reset count since validation loss improved
                self.stop_count = 0
                self.best_epoch = epoch + 1 # set the value of the best epoch to this epoch
            else: # validation loss did not improve
                if self.count >= self.patience - 1: # need to adjust lr
                    lr = lr * self.factor # adjust the learning rate
                    self.stop_count = self.stop_count + 1 # increment stop counter because lr was adjusted
                    self.count = 0 # reset counter
                    tf.keras.backend.set_value(self.model.optimizer.lr, lr) # set the learning rate in the optimizer
                else:
                    self.count = self.count + 1 # increment the patience counter
                if acc > self.highest_tracc:
                    self.highest_tracc = acc

        msg = f'{str(epoch + 1):^3s}/{str(self.epochs):4s} {loss:^9.3f}{acc * 100:^9.3f}{v_loss:^9.5f}{v_acc * 100:^9.3f}{current_lr:^9.5f}{lr:^9.5f}{monitor:^11s}{pimprov:^10.2f}{duration:^8.2f}'
        print(msg)

        if self.stop_count > self.stop_patience - 1: # check if learning rate has been adjusted stop_count times with no improvement
            msg = f' training has been halted at epoch {epoch + 1} after {self.stop_patience} adjustments of learning rate with no improvement'
            print(msg)
            self.model.stop_training = True # stop training

        else:
            if self.ask_epoch != None:
                if epoch + 1 >= self.ask_epoch:
                    msg = 'enter H to halt training or an integer for number of epochs to run then ask again'
                    print(msg)
                    ans = input('')
                    if ans == 'H' or ans == 'h':
                        msg = f'training has been halted at epoch {epoch + 1} due to user input'
                        print(msg)
                        self.model.stop_training = True # stop training
                    else:
                        try:
                            ans = int(ans)
                            self.ask_epoch += ans
                            msg = f' training will continue until epoch ' + str(self.ask_epoch)
                            print(msg)
                            msg = '{0:^8s}{1:^10s}{2:^9s}{3:^9s}{4:^9s}{5:^9s}{6:^9s}{7:^10s}{8:10s}{9:^8s}'.format('Epoch', 'Loss', 'Accuracy', 'V_loss', 'V_acc', 'LR', 'Next LR', 'Monitor', '% Improv', 'Duration')
                            print(msg)
                        except:
                            print('Invalid')

In [14]:
batch_size = 40
epochs = 50
patience = 1
stop_patience = 3
threshold = 0.9
factor = 0.5
freeze = False
ask_epoch = 1
batches = int(np.ceil(len(train_images.labels) / batch_size))


callbacks = [MyCallback(model= model, base_model= base_model, patience= patience,
            stop_patience= stop_patience, threshold= threshold, factor= factor,
            batches= batches, initial_epoch= 0, epochs= epochs, ask_epoch= ask_epoch )]

In [ ]:
history = model.fit(x= train_images, epochs= epochs, verbose= 0, callbacks= callbacks,
                    validation_data= val_images, validation_steps= None, shuffle= True,
                    initial_epoch= 0)

 Epoch     Loss   Accuracy  V_loss    V_acc     LR     Next LR  Monitor  % Improv  Duration
 1 /50    10.451   13.336   7.57511  39.799   0.00100  0.00100  accuracy     0.00   2045.76 
enter H to halt training or an integer for number of epochs to run then ask again
5
 training will continue until epoch 6
 Epoch     Loss   Accuracy  V_loss    V_acc     LR     Next LR  Monitor  % Improv  Duration
 2 /50     6.300   48.285   5.03926  58.824   0.00100  0.00100  accuracy    262.07   127.55 
 3 /50     4.173   68.892   3.69189  68.045   0.00100  0.00100  accuracy    42.68    129.43 
 4 /50     2.924   81.340   2.95901  70.747   0.00100  0.00100  accuracy    18.07    128.27 
 5 /50     2.192   88.425   2.50362  71.807   0.00100  0.00100  accuracy     8.71    127.64 
 6 /50     1.731   93.259   2.27147  71.701   0.00100  0.00100  val_loss     9.27    127.77 
enter H to halt training or an integer for number of epochs to run then ask again
H
training has been halted at epoch 6 due to user inpu

In [ ]:
base_model.trainable = True
for layer in base_model.layers:
    if isinstance(layer, layers.BatchNormalization): # set BatchNorm layers as not trainable
        layer.trainable = False

# let`s see first 10 layers
for l in base_model.layers[:10]:
    print(l.name, l.trainable)

model.compile(
    optimizer=Adam(0.00001), # fine tuning requires very little learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
#model.load_weights('./checkpoints/my_checkpoint')
print(model.summary())
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=20,
    callbacks=[
        EarlyStopping(monitor = "val_loss", # watch the val loss metric
                               patience = 5,
                               restore_best_weights = True), # if val loss decreases for 5 epochs in a row, stop training,
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, mode='min')
    ]
)
model.save_weights('./checkpoints/my_checkpoint')
img_size = (224, 224,3)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(list(train_images.class_indices.keys()))  # to define the number of classes in the dense layer

# create pre-trained model
base_model = tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, weights="imagenet", input_shape=img_shape, pooling='max')

model = Sequential([
    base_model,
    BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
    Dense(1024, kernel_regularizer=regularizers.l2(l=0.016), activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006), activation='relu'),
    Dense(512, kernel_regularizer=regularizers.l2(l=0.016), activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006), activation='relu'),
    Dropout(rate=0.35, seed=123),

    Dense(class_count, activation='softmax')
])

model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.load_weights('./checkpoints/my_checkpoint')
model.summary()

batch_size = 40
epochs = 20
patience = 1
stop_patience = 3
threshold = 0.9
factor = 0.5
freeze = False
ask_epoch = 5
batches = int(np.ceil(len(train_images.labels) / batch_size))


callbacks = [MyCallback(model= model, base_model= base_model, patience= patience,
            stop_patience= stop_patience, threshold= threshold, factor= factor,
            batches= batches, initial_epoch= 0, epochs= epochs, ask_epoch= ask_epoch )]

history = model.fit(x= train_images, epochs= epochs, verbose= 0, callbacks= callbacks,
                    validation_data= val_images, validation_steps= None, shuffle= True,
                    initial_epoch= 0)

input_1 True
rescaling True
normalization True
rescaling_1 True
stem_conv_pad True
stem_conv True
stem_bn False
stem_activation True
block1a_dwconv True
block1a_bn False
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional)  (None, 1536)             10783535  
                                                                 
 batch_normalization (BatchN  (None, 1536)             6144      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 256)               393472    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               51400     
                  

In [27]:
base_model.trainable = True
for layer in base_model.layers:
    if isinstance(layer, layers.BatchNormalization): # set BatchNorm layers as not trainable
        layer.trainable = False

# let`s see first 10 layers
for l in base_model.layers[:10]:
    print(l.name, l.trainable)

model.compile(
    optimizer=Adam(0.00001), # fine tuning requires very little learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
#model.load_weights('./checkpoints/my_checkpoint')
print(model.summary())
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=20,
    callbacks=[
        EarlyStopping(monitor = "val_loss", # watch the val loss metric
                               patience = 5,
                               restore_best_weights = True), # if val loss decreases for 5 epochs in a row, stop training,
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, mode='min')
    ]
)
model.save_weights('/content/drive/MyDrive/DeepLearningModels/e3weight.h5')
img_size = (224, 224,3)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(list(train_images.class_indices.keys()))  # to define the number of classes in the dense layer

# create pre-trained model
base_model = tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, weights="imagenet", input_shape=img_shape, pooling='max')

model = Sequential([
    base_model,
    BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
    Dense(1024, kernel_regularizer=regularizers.l2(l=0.016), activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006), activation='relu'),
    Dense(512, kernel_regularizer=regularizers.l2(l=0.016), activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006), activation='relu'),
    Dropout(rate=0.35, seed=123),

    Dense(class_count, activation='softmax')
])

model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
#model.load_weights('./checkpoints/my_checkpoint')
model.summary()

batch_size = 40
epochs = 20
patience = 1
stop_patience = 3
threshold = 0.9
factor = 0.5
freeze = False
ask_epoch = 5
batches = int(np.ceil(len(train_images.labels) / batch_size))


callbacks = [MyCallback(model= model, base_model= base_model, patience= patience,
            stop_patience= stop_patience, threshold= threshold, factor= factor,
            batches= batches, initial_epoch= 0, epochs= epochs, ask_epoch= ask_epoch )]

history = model.fit(x= train_images, epochs= epochs, verbose= 0, callbacks= callbacks,
                    validation_data= val_images, validation_steps= None, shuffle= True,
                    initial_epoch= 0)

input_8 True
rescaling_14 True
normalization_7 True
rescaling_15 True
stem_conv_pad True
stem_conv True
stem_bn False
stem_activation True
block1a_dwconv True
block1a_bn False
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional)  (None, 1536)             10783535  
                                                                 
 batch_normalization_7 (Batc  (None, 1536)             6144      
 hNormalization)                                                 
                                                                 
 dense_21 (Dense)            (None, 1024)              1573888   
                                                                 
 dense_22 (Dense)            (None, 512)               524800    
                                                                 
 dropout_14 (Dropout)        (None, 512)               0         
          

KeyboardInterrupt: ignored

In [ ]:
model.save_weights('/content/drive/MyDrive/DeepLearningModels/e3weight.h5')

In [ ]:
history = model.fit(x= train_images, epochs= epochs, verbose= 0, callbacks= callbacks,
                    validation_data= val_images, validation_steps= None, shuffle= True,
                    initial_epoch= 0)

 Epoch     Loss   Accuracy  V_loss    V_acc     LR     Next LR  Monitor  % Improv  Duration
 1 /50    15.975   22.752  14.86741  46.052   0.00001  0.00001  accuracy     0.00    246.54 
 2 /50    15.579   27.970  15.00817  43.084   0.00001  0.00001  accuracy    22.93    126.17 
 3 /50    15.249   33.241  14.77019  45.840   0.00001  0.00001  accuracy    18.84    126.08 
 4 /50    14.938   37.465  14.52063  47.854   0.00001  0.00001  accuracy    12.71    126.52 
 5 /50    14.658   42.008  14.28663  50.503   0.00001  0.00001  accuracy    12.12    126.82 
 6 /50    14.379   46.749  14.05863  52.835   0.00001  0.00001  accuracy    11.29    126.18 
 7 /50    14.158   48.735  13.84857  55.485   0.00001  0.00001  accuracy     4.25    126.57 
 8 /50    13.920   51.834  13.64486  57.340   0.00001  0.00001  accuracy     6.36    126.71 
 9 /50    13.713   55.079  13.46105  58.612   0.00001  0.00001  accuracy     6.26    126.51 
10 /50    13.483   57.860  13.28799  59.565   0.00001  0.00001  accurac

UnknownError: ignored

In [26]:
history = model.fit(x= train_images, epochs= epochs, verbose= 0, callbacks= callbacks,
                    validation_data= val_images, validation_steps= None, shuffle= True,
                    initial_epoch= 0)

 Epoch     Loss   Accuracy  V_loss    V_acc     LR     Next LR  Monitor  % Improv  Duration
 1 /50    19.811    0.742  10.18629   0.848   0.00100  0.00100  accuracy     0.00    194.83 
 2 /50     7.525    0.649   6.06539   0.265   0.00100  0.00050  accuracy    -12.50   128.61 
 3 /50     5.790    0.596   5.58617   0.265   0.00050  0.00025  accuracy    -19.64   127.52 
 4 /50     5.519    0.622   5.46301   0.265   0.00025  0.00013  accuracy    -16.07   128.38 
 training has been halted at epoch 4 after 3 adjustments of learning rate with no improvement
training elapsed time was 0.0 hours, 10.0 minutes, 31.26 seconds)


In [16]:
base_model.trainable = True
for layer in base_model.layers:
    if isinstance(layer, layers.BatchNormalization): # set BatchNorm layers as not trainable
        layer.trainable = False

# let`s see first 10 layers
for l in base_model.layers[:10]:
    print(l.name, l.trainable)

model.compile(
    optimizer=Adam(0.00001), # fine tuning requires very little learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
#model.load_weights('./checkpoints/my_checkpoint')
print(model.summary())
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=30,
    callbacks=[
        EarlyStopping(monitor = "val_loss", # watch the val loss metric
                               patience = 5,
                               restore_best_weights = True), # if val loss decreases for 5 epochs in a row, stop training,
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, mode='min')
    ]
)
model.save_weights('/content/drive/MyDrive/DeepLearningModels/e3weight.h5')
img_size = (224, 224,3)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(list(train_images.class_indices.keys()))  # to define the number of classes in the dense layer

# create pre-trained model
base_model = tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, weights="imagenet", input_shape=img_shape, pooling='max')

model = Sequential([
    base_model,
    BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
    Dense(1024, kernel_regularizer=regularizers.l2(l=0.016), activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006), activation='relu'),
    Dense(512, kernel_regularizer=regularizers.l2(l=0.016), activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006), activation='relu'),
    Dropout(rate=0.35, seed=123),

    Dense(class_count, activation='softmax')
])

model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.load_weights('./checkpoints/my_checkpoint')
model.summary()

batch_size = 40
epochs = 50
patience = 1
stop_patience = 3
threshold = 0.9
factor = 0.5
freeze = False
ask_epoch = 20
batches = int(np.ceil(len(train_images.labels) / batch_size))


callbacks = [MyCallback(model= model, base_model= base_model, patience= patience,
            stop_patience= stop_patience, threshold= threshold, factor= factor,
            batches= batches, initial_epoch= 0, epochs= epochs, ask_epoch= ask_epoch )]

history = model.fit(x= train_images, epochs= epochs, verbose= 0, callbacks= callbacks,
                    validation_data= val_images, validation_steps= None, shuffle= True,
                    initial_epoch= 0)

input_5 True
rescaling_8 True
normalization_4 True
rescaling_9 True
stem_conv_pad True
stem_conv True
stem_bn False
stem_activation True
block1a_dwconv True
block1a_bn False
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional)  (None, 1536)             10783535  
                                                                 
 batch_normalization_4 (Batc  (None, 1536)             6144      
 hNormalization)                                                 
                                                                 
 dense_12 (Dense)            (None, 2048)              3147776   
                                                                 
 dropout_8 (Dropout)         (None, 2048)              0         
                                                                 
 dense_13 (Dense)            (None, 2048)              4196352   
            

KeyboardInterrupt: ignored

In [ ]:
!pip install keras-tuner
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB3
from kerastuner import HyperModel, RandomSearch, Objective

# Define your bird species classification dataset and preprocessing here.
# Replace `train_data`, `val_data`, `num_classes`, and other data-related variables accordingly.

# Define the HyperModel for EfficientNetB3
class EfficientNetB3HyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = img_size
        self.num_classes = class_count

    def build(self, hp):
        base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=self.input_shape)

        for layer in base_model.layers:
            layer.trainable = False

        model = tf.keras.Sequential()

        model.add(base_model)
        model.add(layers.GlobalAveragePooling2D())
        model.add(layers.Dense(units=hp.Int('dense_units', min_value=128, max_value=512, step=32), activation='relu'))
        model.add(layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))
        model.add(layers.Dense(units=hp.Int('dense_units', min_value=128, max_value=512, step=32), activation='relu'))
        model.add(layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))
        model.add(layers.Dense(self.num_classes, activation='softmax'))

        return model

# Define the hyperparameter search space
hypermodel = EfficientNetB3HyperModel(input_shape=(224, 224, 3), num_classes=class_count)

# Define the objective function
objective = Objective('val_accuracy', direction='max')

# Initialize the RandomSearch tuner
tuner = RandomSearch(
    hypermodel,
    objective=objective,
    seed=42,
    max_trials=10,
    directory='my_dir',
    project_name='bird_species_classification'
)

# Perform hyperparameter tuning
tuner.search(train_images, epochs=10, validation_data=val_images)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=10)[0]
print("Best Hyperparameters:")
print(best_hps)

# Build and compile the model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)
best_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_hps.get('learning_rate')),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

# Train the model with the best hyperparameters
best_model.fit(train_images, epochs=50, validation_data=val_images)



In [ ]:
!pip install tensorflow
!pip install keras-tuner

!pip install scikit-optimize



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 1.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB3

# Create the EfficientNetB3 model
def create_model(img_shape, class_count, dense_units=512, dropout_rate=0.3):
    base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=img_shape, pooling='max')
    for layer in base_model.layers:
        layer.trainable = False

    model = tf.keras.Sequential([
        base_model,
        layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
        layers.Dense(units=dense_units, activation='relu'),
        layers.Dropout(rate=dropout_rate),
        layers.Dense(class_count, activation='softmax')
    ])

    return model
model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])



model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional)  (None, 1536)             10783535  
                                                                 
 batch_normalization_1 (Batc  (None, 1536)             6144      
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 512)               786944    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                      

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import OneHotEncoder
# Define hyperparameters for tuning
param_grid = {
    'epochs': [5, 10, 15],
    'learning_rate': [0.001, 0.0001, 0.00001],
    'kernel_regularizer': [None, l2(0.001)],  # None or L2 regularization
    'dropout_rate': [0.0, 0.2, 0.5],  # Dropout rate for the Dense layer
}

# Create GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# Perform hyperparameter search
grid_search.fit(train_images, val_images)

# Print the best hyperparameters and corresponding score
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
test_loss, test_accuracy = best_model.evaluate(train_images, val_images)

print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")

